# Coursework 2: Image segmentation

In this coursework you will develop and train a convolutional neural network for brain tumour image segmentation. Please read both the text and the code in this notebook to get an idea what you are expected to implement. Pay attention to the missing code blocks that look like this:

```
### Insert your code ###
...
### End of your code ###
```

## What to do?

* Complete and run the code using `jupyter-lab` or `jupyter-notebook` to get the results.

* Export (File | Save and Export Notebook As...) the notebook as a PDF file, which contains your code, results and answers, and upload the PDF file onto [Scientia](https://scientia.doc.ic.ac.uk).

* Instead of clicking the Export button, you can also run the following command instead: `jupyter nbconvert coursework.ipynb --to pdf`

* If Jupyter complains about some problems in exporting, it is likely that pandoc (https://pandoc.org/installing.html) or latex is not installed, or their paths have not been included. You can install the relevant libraries and retry.

* If Jupyter-lab does not work for you at the end, you can use Google Colab to write the code and export the PDF file.

## Dependencies

You need to install Jupyter-Lab (https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html) and other libraries used in this coursework, such as by running the command:
`pip3 install [package_name]`

## GPU resource

The coursework is developed to be able to run on CPU, as all images have been pre-processed to be 2D and of a smaller size, compared to original 3D volumes.

However, to save training time, you may want to use GPU. In that case, you can run this notebook on Google Colab. On Google Colab, go to the menu, Runtime - Change runtime type, and select **GPU** as the hardware acceleartor. At the end, please still export everything and submit as a PDF file on Scientia.


In [1]:
# Import libraries
# These libraries should be sufficient for this tutorial.
# However, if any other library is needed, please install by yourself.

import tarfile
import imageio as imageio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
import numpy as np
import time
import os
import random
import matplotlib.pyplot as plt
from matplotlib import colors

## 1. Download and visualise the imaging dataset.

The dataset is curated from the brain imaging dataset in [Medical Decathlon Challenge](http://medicaldecathlon.com/). To save the storage and reduce the computational cost for this tutorial, we extract 2D image slices from T1-Gd contrast enhanced 3D brain volumes and downsample the images.

The dataset consists of a training set and a test set. Each image is of dimension 120 x 120, with a corresponding label map of the same dimension. There are four number of classes in the label map:

- 0: background
- 1: edema
- 2: non-enhancing tumour
- 3: enhancing tumour

In [ ]:
# Download the dataset
!pwd
!wget https://www.dropbox.com/s/zmytk2yu284af6t/Task01_BrainTumour_2D.tar.gz

# Unzip the '.tar.gz' file to the current directory
datafile = tarfile.open('Task01_BrainTumour_2D.tar.gz')
datafile.extractall()
datafile.close()

## Visualise a random set of 4 training images along with their label maps.

Suggested colour map for brain MR image:
```
cmap = 'gray'
```

Suggested colour map for segmentation map:
```
cmap = colors.ListedColormap(['black', 'green', 'blue', 'red'])
```

In [ ]:
### Insert your code ###
import glob

imgs = glob.glob("Task01_BrainTumour_2D/training_images/*")
short = random.sample(imgs, 4)
for i, img in enumerate(short):
    image = imageio.imread(img)
    plt.subplot(2, 2, i+1)
    plt.imshow(image, cmap='gray')
    plt.gcf().set_size_inches(8, 8)
### End of your code ###

## 2. Implement a dataset class.

It can read the imaging dataset and get items, pairs of images and label maps, as training batches.

In [ ]:
def normalise_intensity(image, thres_roi=1.0):
    """ Normalise the image intensity by the mean and standard deviation """
    # ROI defines the image foreground
    val_l = np.percentile(image, thres_roi)
    roi = (image >= val_l)
    mu, sigma = np.mean(image[roi]), np.std(image[roi])
    eps = 1e-6
    image2 = (image - mu) / (sigma + eps)
    return image2


class BrainImageSet(Dataset):
    """ Brain image set """
    def __init__(self, image_path, label_path='', deploy=False):
        self.image_path = image_path
        self.deploy = deploy
        self.images = []
        self.labels = []

        image_names = sorted(os.listdir(image_path))
        for image_name in image_names:
            # Read the image
            image: np.ndarray = imageio.imread(os.path.join(image_path, image_name))
            # print(image.shape)
            self.images += [image]

            # Read the label map
            if not self.deploy:
                label_name = os.path.join(label_path, image_name)
                label = imageio.imread(label_name)
                self.labels += [label]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Get an image and perform intensity normalisation
        # Dimension: XY
        image = normalise_intensity(self.images[idx])

        # Get its label map
        # Dimension: XY
        label = self.labels[idx]
        return image, label

    def get_random_batch(self, batch_size):
        # Get a batch of paired images and label maps
        # Dimension of images: NCXY
        # Dimension of labels: NXY
        images, labels = [], []

        ### Insert your code ###
        n = self.__len__()
        ixs = np.arange(0, n, 1)
        rng = np.random.default_rng()
        selected = rng.choice(a=ixs, size=batch_size, replace=False)
        images, labels = np.array(self.images), np.array(self.labels)
        images, labels = images[selected], labels[selected]
        for i in range(len(images)):
            images[i] = normalise_intensity(images[i])
        images = images[:, np.newaxis, :, :]
        ### End of your code ###
        return images, labels

## 3. Build a U-net architecture.

You will implement a U-net architecture. If you are not familiar with U-net, please read this paper:

[1] Olaf Ronneberger et al. [U-Net: Convolutional networks for biomedical image segmentation](https://link.springer.com/chapter/10.1007/978-3-319-24574-4_28). MICCAI, 2015.

For the first convolutional layer, you can start with 16 filters. We have implemented the encoder path. Please complete the decoder path.

In [ ]:
""" U-net """
class UNet(nn.Module):
    def __init__(self, input_channel=1, output_channel=1, num_filter=16):
        super(UNet, self).__init__()

        # BatchNorm: by default during training this layer keeps running estimates
        # of its computed mean and variance, which are then used for normalization
        # during evaluation.

        # Encoder path
        n = num_filter  # 16
        self.conv1 = nn.Sequential(
            nn.Conv2d(input_channel, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU()
        )

        n *= 2  # 32
        self.conv2 = nn.Sequential(
            nn.Conv2d(int(n / 2), n, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU()
        )

        n *= 2  # 64
        self.conv3 = nn.Sequential(
            nn.Conv2d(int(n / 2), n, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU()
        )

        n *= 2  # 128
        self.conv4 = nn.Sequential(
            nn.Conv2d(int(n / 2), n, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU()
        )
        
        # Decoder path
        ### Insert your code ###
        n *= 2 # 256
        self.bot = nn.Sequential(
            nn.Conv2d(int(n / 2), n, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.ConvTranspose2d(n, int(n / 2), kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(int(n / 2)),
            nn.ReLU()
        )

        n //= 2  # 128
        self.deco1 = nn.Sequential(
            nn.Conv2d(2 * n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.ConvTranspose2d(n, int(n / 2), kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(int(n / 2)),
            nn.ReLU()
        )

        n //= 2  # 64
        self.deco2 = nn.Sequential(
            nn.Conv2d(2 * n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.ConvTranspose2d(n, int(n / 2), kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(int(n / 2)),
            nn.ReLU()
        )

        n //= 2  # 32
        self.deco3 = nn.Sequential(
            nn.Conv2d(2 * n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.ConvTranspose2d(n, int(n / 2), kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(int(n / 2)),
            nn.ReLU()
        )

        n //= 2  # 16
        self.deco4 = nn.Sequential(
            nn.Conv2d(2 * n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, n, kernel_size=3, padding=1),
            nn.BatchNorm2d(n),
            nn.ReLU(),
            nn.Conv2d(n, output_channel, kernel_size=1),
            nn.BatchNorm2d(output_channel),
            nn.ReLU()
        )
        ### End of your code ###

    def forward(self, x):
        # Use the convolutional operators defined above to build the U-net
        # The encoder part is already done for you.
        # You need to complete the decoder part.
        # Encoder
        x = self.conv1(x)
        conv1_skip = x

        x = self.conv2(x)
        conv2_skip = x

        x = self.conv3(x)
        conv3_skip = x

        x = self.conv4(x)

        # Decoder
        ### Insert your code ###
        conv4_skip = x
        x = self.bot(x)

        x = torch.cat((x, conv4_skip), dim=1)
        x = self.deco1(x)

        x = torch.cat((x, conv3_skip), dim=1)
        x = self.deco2(x)

        x = torch.cat((x, conv2_skip), dim=1)
        x = self.deco3(x)

        x = torch.cat((x, conv1_skip), dim=1)
        x = self.deco4(x)
        ### End of your code ###
        return x

## 4. Train the segmentation model.

In [ ]:
from tqdm import tqdm
import torchvision.transforms as T

# CUDA device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: {0}'.format(device))

# Build the model
num_class = 4
model = UNet(input_channel=1, output_channel=num_class, num_filter=16)
model = model.to(device)
params = list(model.parameters())

model_dir = 'saved_models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Optimizer
optimizer = optim.Adam(params, lr=1e-3)

# Segmentation loss
criterion = nn.CrossEntropyLoss()

# Datasets
train_set = BrainImageSet('Task01_BrainTumour_2D/training_images', 'Task01_BrainTumour_2D/training_labels')
test_set = BrainImageSet('Task01_BrainTumour_2D/test_images', 'Task01_BrainTumour_2D/test_labels')

# Train the model
# Note: when you debug the model, you may reduce the number of iterations or batch size to save time.
num_iter = 15_000
num_switch = 10_000
train_batch_size = 16
eval_batch_size = 16
jitter = T.ColorJitter(brightness=.5, hue=.3, contrast=1, saturation=1)
start = time.time()
for it in tqdm(range(1, 1 + num_iter)):
    # Set the modules in training mode, which will have effects on certain modules, e.g. dropout or batchnorm.
    start_iter = time.time()
    model.train()

    # Get a batch of images and labels
    images, labels = train_set.get_random_batch(train_batch_size)
    images, labels = torch.from_numpy(images), torch.from_numpy(labels)
    images, labels = images.to(device, dtype=torch.float32), labels.to(device, dtype=torch.long)
    if it >= num_switch:
      for i in range(len(images)):
        images[i] = jitter(images[i])
    logits = model(images)

    # Perform optimisation and print out the training loss
    ### Insert your code ###
    optimizer.zero_grad()
    loss = criterion(logits, labels)
    loss.backward()
    optimizer.step()
    ### End of your code ###

    # Evaluate
    if it % (num_iter // 100) == 0:
        model.eval()
        # Disabling gradient calculation during reference to reduce memory consumption
        with torch.no_grad():
            # Evaluate on a batch of test images and print out the test loss
            ### Insert your code ###
            images, labels = test_set.get_random_batch(train_batch_size)
            images, labels = torch.from_numpy(images), torch.from_numpy(labels)
            images, labels = images.to(device, dtype=torch.float32), labels.to(device, dtype=torch.long)
            logits = model(images)
            loss_test = criterion(logits, labels)
            print(f'{loss.item():.3f}', f'{loss_test.item():.3f}')
        
    if it == num_iter:
      model.eval()
      # Disabling gradient calculation during reference to reduce memory consumption
      with torch.no_grad():
          # Evaluate on a batch of test images and print out the test loss
          ### Insert your code ###
          images, labels = test_set.get_random_batch(len(test_set))
          images, labels = torch.from_numpy(images), torch.from_numpy(labels)
          images, labels = images.to(device, dtype=torch.float32), labels.to(device, dtype=torch.long)
          logits = model(images)
          loss_test = criterion(logits, labels)
          print('Loss on whole test set:', f'{loss_test.item():.3f}')
          ### End of your code ###

    # Save the model
    if it % 5000 == 0:
        torch.save(model.state_dict(), os.path.join(model_dir, 'model_{0}.pt'.format(it)))
print('Training took {:.3f}s in total.'.format(time.time() - start))

## 5. Deploy the trained model to a random set of 4 test images and visualise the automated segmentation.

You can show the images as a 4 x 3 panel. Each row shows one example, with the 3 columns being the test image, automated segmentation and ground truth segmentation.

In [ ]:
### Insert your code ###
images, labels = test_set.get_random_batch(4)
images, labels = torch.from_numpy(images), torch.from_numpy(labels)
images, labels = images.to(device, dtype=torch.float32), labels.to(device, dtype=torch.long)
model.train(False)
logits = model(images)
results = torch.argmax(logits, 1)
foo = zip(images, results, labels)

cmap = colors.ListedColormap(['white', 'green', 'yellow', 'red'])

for i, (test, auto, ground) in enumerate(foo):
    test = torch.squeeze(test)
    plt.subplot(4, 3, 3 * i + 1)
    plt.imshow(test.cpu(), cmap='gray')
    plt.gcf().set_size_inches(8, 8)

    auto = torch.squeeze(auto)
    plt.subplot(4, 3, 3 * i + 2)
    plt.imshow(auto.cpu(), cmap=cmap)
    plt.gcf().set_size_inches(8, 8)

    ground = torch.squeeze(ground)
    plt.subplot(4, 3, 3 * i + 3)
    plt.imshow(ground.cpu(), cmap=cmap)
    plt.gcf().set_size_inches(8, 8)
### End of your code ###

## 6. Discussion. Does your trained model work well? How would you improve this model so it can be deployed to the real clinic?

**Evaluation**

The model works reasonably well. The mean cross-entropy loss evaluated on the whole test set (code cell below) is 0.061. In the majority of cases it places the non-background classes (edema and tumours) in the right place. However, the shape is usually not the same as the ground truth and the non-background classes are a bit shifted. The model often confuses tumours and edema. Sometimes the model doesn't detect a tumour where there actually is one and vice versa.

**Future improvements**

In the future the following improvements could be made to increase the accuracy:
* Data augmentation + more iterations (grid distortion, and elastic transform). The current model uses augmentation in the form of color tranformation (ColorJitter) but other types could be added.
* More training data
* Use UNet++ architecture (insert convolution layers between skip connections)
* Add instance segmentation to differentiate between individual tumour clusters
* Add some form of early stopping, L2 regularisation or weight pruning to prevent overfitting


# Evaluate accuracy on whole test set

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: {0}'.format(device))

num_class = 4
model = UNet(input_channel=1, output_channel=num_class, num_filter=16)
model.load_state_dict(torch.load('saved_models/model_15000.pt'))
model = model.to(device)
model.eval()

# Disabling gradient calculation during reference to reduce memory consumption
with torch.no_grad():
    # Evaluate on a batch of test images and print out the test loss
    ### Insert your code ###
    print(len(test_set))
    images, labels = test_set.get_random_batch(len(test_set))
    images, labels = torch.from_numpy(images), torch.from_numpy(labels)
    images, labels = images.to(device, dtype=torch.float32), labels.to(device, dtype=torch.long)
    logits = model(images)
    loss_test = criterion(logits, labels)
    print('Loss on whole test set:', f'{loss_test.item():.3f}')
    ### End of your code ###

In [43]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: {0}'.format(device))

num_class = 4
model = UNet(input_channel=1, output_channel=num_class, num_filter=16)
model.load_state_dict(torch.load('saved_models/model_15000.pt'))
model = model.to(device)
model.eval()

# Disabling gradient calculation during reference to reduce memory consumption
with torch.no_grad():
    # Evaluate on a batch of test images and print out the test loss
    ### Insert your code ###
    print(len(test_set))
    images, labels = test_set.get_random_batch(len(test_set))
    images, labels = torch.from_numpy(images), torch.from_numpy(labels)
    images, labels = images.to(device, dtype=torch.float32), labels.to(device, dtype=torch.long)
    logits = model(images)
    loss_test = criterion(logits, labels)
    print('Loss on whole test set:', f'{loss_test.item():.3f}')
    ### End of your code ###

Device: cuda
730
Loss on whole test set: 0.065
